## Check data for algae blooms identification.

The idea is to take the modeled data and train a machine learning (ML) model on that data, then try to use on the observational data.
The reason - models can't predict very well the exact time and location of algae blooms but they reproduce the physics/biogeochemistry of it.
Thus, the intuition to check is that a ML model trained on modelled data will be able to predict blooms on observational data.

In [ ]:
import os
from functools import partial
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from blooms_ml.utils import (
    labeling,
    normalize_df,
)

sns.set_style("whitegrid")

In [ ]:
VARIABLES = [*["N1_p", "N3_n", "N5_s", "P1_c"], *[str(i) for i in range(1, 26)]]

Open the prepared from ROHO800 model output data and label it

In [ ]:
datadir = f"{Path.home()}/data_ROHO"
df = pd.read_parquet(os.path.join(datadir, "roho800_weekly_average.parquet"))

In [ ]:
df = df.groupby(["station", "s_rho"]).apply(labeling, include_groups=False)
df = df.reset_index().drop(columns="level_2")

In [ ]:
def to_differences(df):
    df = df.reset_index(drop=True)
    df_diff1 = (df[VARIABLES].diff(periods=1))[2:]
    df_diff1 = df_diff1.rename(columns=lambda x: x + "_diff1")
    df_diff2 = (df[VARIABLES].diff(periods=2))[2:]
    df_diff2 = df_diff2.rename(columns=lambda x: x + "_diff2")
    df_diff = pd.concat([df_diff1, df_diff2], axis=1)
    df_diff[["ocean_time", "label"]] = df[["ocean_time", "label"]]
    return df_diff

In [ ]:
df_diff = df.groupby(["station", "s_rho"]).apply(to_differences, include_groups=False)
df_diff = df_diff.reset_index().drop(columns="level_2")

In [ ]:
df_diff = df_diff.groupby(["station", "s_rho"]).apply(
    partial(normalize_df, columns_slice=slice(0, -2)), include_groups=False
)
df_diff = df_diff.reset_index().drop(columns="level_2")

In [ ]:
df_diff

Plot a station

In [ ]:
def plot_year(features, labels, par):
    _, ax = plt.subplots(figsize=(20, 3))
    ax.plot(features[par].index, features[par])
    ax.plot(labels.index, labels[par], "r.")

In [ ]:
station = 0
s_rho = -0.02
year = "2007"

In [ ]:
df_station = df[df["station"] == station][df["s_rho"] == s_rho]
df_station.set_index("ocean_time", inplace=True)

In [ ]:
df_station_diff = df_diff[df_diff["station"] == station][df_diff["s_rho"] == s_rho]
df_station_diff.set_index("ocean_time", inplace=True)

In [ ]:
df_year = df_station.loc[year]
df_label_year = df_year[df_year["label"] == 1]

df_year_diff = df_station_diff.loc[year]
df_label_year_diff = df_year_diff[df_year_diff["label"] == 1]

In [ ]:
plot_year(df_year, df_label_year, "P1_c")

In [ ]:
plot_year(df_year_diff, df_label_year_diff, "P1_c_diff1")

In [ ]:
plot_year(df_year_diff, df_label_year_diff, "P1_c_diff2")

Unsupervised

In [ ]:
import time

from sklearn.decomposition import PCA

In [ ]:
df_surface = df_diff[df_diff["s_rho"] == -0.02]

In [ ]:
df_surface = df_surface.drop(columns=["station", "s_rho", "ocean_time"])
df_surface = df_surface.reset_index(drop=True)

In [ ]:
df_surface

In [ ]:
X = df_surface.drop("label", axis=1)
y = df_surface["label"]

In [ ]:
# PCA
t0 = time.time()
pca = PCA(n_components=3, random_state=42)
X_reduced_pca = pca.fit_transform(X.values)
t1 = time.time()
print(f"PCA took {t1 - t0:.2} s")

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
import matplotlib.patches as mpatches

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(8, 8))
f.suptitle("Clusters using Dimensionality Reduction", fontsize=14)

blue_patch = mpatches.Patch(color="#0A0AFF", label="No bloom")
red_patch = mpatches.Patch(color="#AF0000", label="bloom")

# PCA scatter plot
ax.scatter(X_reduced_pca[:, 0], X_reduced_pca[:, 1], c=(y == 0), cmap="coolwarm", label="No bloom", linewidths=2)
ax.scatter(X_reduced_pca[:, 0], X_reduced_pca[:, 1], c=(y == 1), cmap="coolwarm", label="bloom", linewidths=2)
ax.set_title("PCA", fontsize=14)

ax.grid(True)

ax.legend(handles=[blue_patch, red_patch])

plt.show()

Classification

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df_input

In [ ]:
X = df_input.drop(columns=["station", "ocean_time", "s_rho", "P1_c", "P1_netPI", "label"])
y = df_input["label"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [ ]:
classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
}

In [ ]:
for key, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    training_score = cross_val_score(classifier, X_test, y_test, cv=5)
    print(
        "Classifiers: ",
        classifier.__class__.__name__,
        "Has a training score of",
        round(training_score.mean(), 2) * 100,
        "% accuracy score",
    )

In [ ]:
log_reg_sm = LogisticRegression()

In [ ]:
log_reg_sm.fit(X_train, y_train)

In [ ]:
y_pred_log_reg = log_reg_sm.predict(X_test)

In [ ]:
labels = ["No bloom", "bloom"]
plot_confusion_matrix(confusion_matrix(y_test, y_pred_log_reg), labels, title="Confusion Matrix", cmap=plt.cm.Reds)